In [ ]:
import pandas as pd

Set your original coding sequence

In [1]:
original_dna ="ATGTACGAAAGATGTGAAGTTTCTTTGTTGGCGATTTTCTTGCAGCGCGATCGCGGTCGGAAAGTAATTTGGCTCACGGTTGGACTCATAACGGTGATATTGGTAATAATAAAGATTTCATCCAGCAAGTCCACCGCCACAGACCTTCAGAAAGTATTGAAGAATGCCAATATTTTACCGCAGGACGTGATCAATTACAATAGCAGAAAAGTAACGGATGAACTCGCTTCAAAGCTGGACGAGATTCAAAAGAAGTATCTCTCGAAGCAAGATGATAGAATTAGCAAGCTCGAAGCTGAACGGGCAGATCTACTGGAACAGGTTAGATTTCTAAGGAACCCCCCTGCAGGATCAAGTTTAAGAGAAAAATTGGCTTATCTGTTTCCTTATAATGAAAACGGCAAATTCCCTGCTTATATATGGCAAACATGGAAGTATGGCTTGAATGACGATCGGTTTGGAGAAAAGTTCAAAGAAGGCGAAACTCAGTGGGCTTCGAAGAATCCTGGTTTCGTTCATGAGTTGTTTAACGATGATACTTCCGGTGTGTTTATTCACCATCTGTATATCAATGTTCCAGAAGTGATCAAAGCATACGAGCTGCTTCCCAACATAATCTTGAAAATGGACTTCTTCAGATATTTGGTTTTATACGCCAAGGGAGGTGTCTATGCAGACGTTGATACTATGCCTCTTCAGCCTGTACCAAACTGGATTCCTGAAAATGTCTCCCCAAAAAGCATCGGGATGATCATTGGAATACAAAACGATGCTAACAATCCAGATTGGAAAAAAGATTACGTACATCGTCTACAATTTTCCAATTGGTGTATTCAAGCGAAGCCTGGTCACCCAATCTTGAGAGAATTAATCGCCAAAATCACAGAGGATACACTGCAACGAGCCGAGTCAAACTCACTGGAACTAGCTGACATTAGCGAAGAAGGCGGCCTGTCTGATAAGAATTTGTCCATTATGCAATGGACGGGTACTGGTATTTTTACAGATGCCATATTTACCTATTTTAATGACTACATTCAAAGTAGTATCTATACCAAAGTTACTTGGAAAGAATTCTCCAAATTGAGAAAGCCCAAGCTTGTCAGTGATGTATTGGTACTGCCGATTATCAGCTTCTCGGCCGGTGCAGGTAGTGGAAAATCGACTGAACTGAACGATCCCTTAGCATTCGTACAACATTATTTTGAAAGATTACATAACGACAACCACTAA"

Read the codon table and encode it for DNA

In [10]:
codon_table = pd.read_csv("data\dna\codon_usage_pichia_pastoris.csv")
codon_table["triplet"] = codon_table["triplet"].str.replace("U", "T")

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\326po\AppData\Local\Temp\ipykernel_19784\1831397788.py:1: SyntaxWarning: invalid escape sequence '\d'
  codon_table = pd.read_csv("data\dna\codon_usage_pichia_pastoris.csv")


Include alternatives in the table

In [15]:
codon_alternatives = (codon_table
    .sort_values(by="fraction",ascending=False)
    .groupby("amino_acid")["triplet"]
    .agg(list).to_dict())

def get_codon_alternatives(row:pd.Series) -> list:
    """Include alternatives for each codon in the table.
    If no alternatives, return the triplet itself."""
    amino_acid, triplet = row[["amino_acid", "triplet"]]
    if len(codon_alternatives[amino_acid]) > 1:
        return [cod for cod in codon_alternatives[amino_acid] if cod != triplet]
    else:
        return [triplet]

codon_table["alternatives"] = codon_table.apply(get_codon_alternatives, axis=1)
codon_table


,triplet,amino_acid,fraction,frequency_per_thousand,count,alternatives
0,TTT,F,0.54,24.1,1963,[TTC]
1,TCT,S,0.29,24.4,1983,"[TCC, TCA, AGT, TCG, AGC]"
2,TAT,Y,0.47,16.0,1300,[TAC]
3,TGT,C,0.64,7.7,626,[TGC]
4,TTC,F,0.46,20.6,1675,[TTT]
...,...,...,...,...,...,...
59,GGA,G,0.33,19.1,1550,"[GGT, GGC, GGG]"
60,GTG,V,0.19,12.3,998,"[GTT, GTC, GTA]"
61,GCG,A,0.06,3.9,314,"[GCT, GCC, GCA]"
62,GAG,E,0.44,29.0,2360,[GAA]


In [16]:
def recode_sequence (sequence:str, codon_table:pd.DataFrame)->str:
    """Encode the least homologous sequence (use new codon when available)."""
    new_sequence =""
    for start in range(0, len(original_dna), 3):
        codon = original_dna[start:start + 3]

        new_codon = codon_table[codon_table["triplet"] == codon]["alternatives"].iloc[0]

        new_sequence += new_codon[0]
    return new_sequence

new_sequence = recode_sequence(original_dna, codon_table)
print(f"Original sequence: {original_dna}")
print(f"New sequence: {new_sequence}")

Original sequence: ATGTACGAAAGATGTGAAGTTTCTTTGTTGGCGATTTTCTTGCAGCGCGATCGCGGTCGGAAAGTAATTTGGCTCACGGTTGGACTCATAACGGTGATATTGGTAATAATAAAGATTTCATCCAGCAAGTCCACCGCCACAGACCTTCAGAAAGTATTGAAGAATGCCAATATTTTACCGCAGGACGTGATCAATTACAATAGCAGAAAAGTAACGGATGAACTCGCTTCAAAGCTGGACGAGATTCAAAAGAAGTATCTCTCGAAGCAAGATGATAGAATTAGCAAGCTCGAAGCTGAACGGGCAGATCTACTGGAACAGGTTAGATTTCTAAGGAACCCCCCTGCAGGATCAAGTTTAAGAGAAAAATTGGCTTATCTGTTTCCTTATAATGAAAACGGCAAATTCCCTGCTTATATATGGCAAACATGGAAGTATGGCTTGAATGACGATCGGTTTGGAGAAAAGTTCAAAGAAGGCGAAACTCAGTGGGCTTCGAAGAATCCTGGTTTCGTTCATGAGTTGTTTAACGATGATACTTCCGGTGTGTTTATTCACCATCTGTATATCAATGTTCCAGAAGTGATCAAAGCATACGAGCTGCTTCCCAACATAATCTTGAAAATGGACTTCTTCAGATATTTGGTTTTATACGCCAAGGGAGGTGTCTATGCAGACGTTGATACTATGCCTCTTCAGCCTGTACCAAACTGGATTCCTGAAAATGTCTCCCCAAAAAGCATCGGGATGATCATTGGAATACAAAACGATGCTAACAATCCAGATTGGAAAAAAGATTACGTACATCGTCTACAATTTTCCAATTGGTGTATTCAAGCGAAGCCTGGTCACCCAATCTTGAGAGAATTAATCGCCAAAATCACAGAGGATACACTGCAACGAGCCGAGTCAAACTCACTGGAACTAGCTGACATTAGCGAAGAAGGCGGCCTGTCTGATAAGAATTTGTCCATTATGCAA

In [17]:
similarity = 0
for i in range(len(original_dna)):
    if original_dna[i] == new_sequence[i]:
        similarity += 1
print(f"Similarity: {similarity / len(original_dna) * 100:.2f}%")

Similarity: 63.02%


Compare codon usage goodness.

In [ ]:
def get_frequency(dna_seq:str)-> float:
    """Return average codon frequency per thousand of the sequence."""
    frequency = 0
    for start in range(0, len(dna_seq), 3):
        codon = dna_seq[start:start + 3]
        frequency += codon_table[codon_table["triplet"] == codon]["frequency_per_thousand"].iloc[0]
    return frequency/ len(dna_seq) * 3

original_frequency = get_frequency(original_dna)
new_frequency = get_frequency(new_sequence)

print(f"Original frequency: {original_frequency:.2f} per thousand")
print(f"New frequency: {new_frequency:.2f} per thousand")

Original frequency: 21.02 per thousand
New frequency: 23.58 per thousand


Check goodness of codon optimised sequence

In [9]:
optimised_sequence = "ATGTATGAAAGATGTGAAGTTTCTCTCCTTGCTATATTTTTGCAACGTGATCGAGGAAGGAAAGTAATTTGGTTAACGGTTGGACTTATAACTGTGATATTAGTTATAATAAAGATTTCATCCTCCAAATCCACAGCTACAGACTTACAGAAAGTATTGAAGAATGCGAATATATTACCACAAGATGTCATCAATTATAATTCAAGAAAAGTAACCGATGAATTGGCTTCAAAACTTGATGAGATTCAAAAAAAGTATCTCAGTAAACAAGATGATAGAATTTCTAAATTGGAAGCTGAAAGAGCCGATCTATTGGAACAAGTTAGATTTTTGAGGAATCCTCCTGCTGGTTCTAGTTTAAGAGAAAAATTGGCTTATCTATTTCCTTATAATGAAAACGGGAAATTTCCAGCATATATTTGGCAAACTTGGAAATATGGCTTAAATGATGATCGTTTTGGAGAAAAATTCAAAGAAGGCGAAACTCAGTGGGCTTCAAAAAATCCAGGTTTTGTTCATGAACTTTTTAATGATGATACTTCTGGTGTGTTTATTCACCATTTGTATATCAATGTTCCAGAAGTAATTAAAGCATATGAATTGCTTCCCAATATAATATTGAAAATGGATTTTTTTAGATATTTGGTTTTATACGCCAAGGGAGGTGTTTATGCAGACGTTGATACTATGCCTCTTCAACCTGTGCCAAATTGGATTCCAGAAAATGTTTCGCCTAAATCTATCGGAATGATTATAGGAATACAAAACGATGCTAATAATCCAGATTGGAAAAAAGATTACGTACATAGATTACAATTTAGTAATTGGTGTATTCAAGCTAAACCTGGACATCCAATATTGAGAGAATTAATCGCCAAAATTACAGAGGATACCTTACAACGTGCCGAGTCAAATTCTTTAGAACTAGCTGATATTAGTGAAGAAGGTGGTTTATCTGATAAGAATCTTTCCATAATGCAATGGACTGGTACAGGAATTTTTACAGACGCAATATTTACTTATTTTAATGATTATATTCAAAGTAGTATTTATACCAAAGTTACATGGAAAGAATTTTCCAAATTGAGAAAACCGAAGCTAGTCTCAGATGTATTGGTATTACCTATTATTAGCTTCTCTGCCGGTGCAGGTTCTGGTAAATCGACTGAACTGAACGATCCTTTAGCATTTGTACAACATTATTTTGAAAGATTACATAATGACAACCACTAA"
pichia_optimised_goodness = get_frequency(optimised_sequence)
pichia_optimised_goodness

np.float64(22.160340632603415)